<a href="https://colab.research.google.com/github/Crone06101995/data_dump/blob/main/01_Datenaufbereitung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **01. Datenaufbereitung**

---
---

## Daten Einlesen

Die artifizielle CSV soll nun in dieses Skript **eingebunden** werden. \
Dafür ist zuerst der Import von Gosling nötig!

**Anmerkung:**\
Google Colaboratory ist ein Visualizer von *Jupyter-Notebooks* und funktioniert daher sehr ähnlich.\
Jedes *Colab* besteht aus Text-und Code-Zellen. Jeglicher Code in Python funktioniert in den Code-Zellen. In den Text-Zellen funktioniert jegliche Markdown-Systematik.\
In unserem Fall dienen *Google Colabs* als interaktives Tutorial mit leichter Handhabe von Hands-On Aufgaben.

In [ ]:
!pip install gosling==0.0.9
import gosling as gos

Gosling arbeitet auf Daten, die durch einen **HTTP-Zugang** öffentlich sind.
Die CSV wurde in einem Git-Repository hochgeladen und als 'Raw'-Darstellung unter einem HTTP-Link hochgestellt. \
Es ist möglich, lokale Daten in Gosling einzuspeisen. Doch dafür muss ein **HiGlass Server** aufgestellt werden. Der 'Workaround', durch Hochladen der Daten zu GitHub-Repositories, scheint bis jetzt die beste Lösung für das Problem zu sein.

Die Daten werden nun unter der Variable *data* abgespeichert und mit der Funktion *gos.csv* eingelesen.

Auch BED-Daten werden mit der '.csv' Funktion geladen. Für andere Datenformate sind andere Methoden von Nöten. Genauere Information dieser Methoden ist hier zu finden:

[Gosling-Data-Formate](http://gosling-lang.org/docs/data#csv-no-higlass-server) \
[Gos-API-Funktionen](https://gosling-lang.github.io/gos/user_guide/API.html)

In [ ]:
data_url = "https://raw.githubusercontent.com/Crone06101995/data_dump/main/artificial_barley_data.csv"

# Einbindung der Daten mithilfe der .csv Funktion
data = gos.csv(
    url=data_url,
    separator=";",
    chromosomeField="Chromosome", # the column containing chromosome names
    genomicFields=["chromStart", "chromEnd"], # fields with (relative) genomic coordinates
)

# JSON unserer Daten
data

**Anmerkung:**
Unter der genauen Betrachtung des Datensatzes fällt auf, dass bestimmte Eigenschaften biologisch und systematisch keinen Sinn machen. Sie sind lediglich auf die Bedienbarkeit angepasst.

Die Daten sind nun eingelesen und sind zur Weiterverarbeitung bereit.\
Goslings zwei wichtigstens Funktionen sind **gos.Track** und **gos.View**.
Diese Kernkomponenten sind Element jeder Visualisierung in Gosling.

*gos.Track* transformiert Daten und setzt sie in visualisierbare Zusammenhänge. Hier werden zum Beispiel die Spalten und Datentypen für die Achsen oder auch die Visualisierungsform (Bar/Scatter/Histogram) gesetzt. 

<img src="https://github.com/gosling-lang/gos-example/blob/main/notebooks/tracks-views.jpeg?raw=1"  width="70%">


*gos.view* setzt die einzelnen Tracks in einem Zusammenhang. Die Visualisierung mehrerer Tracks erfolgt durch die Gruppierung und Darstellungoptionen mit *gos.view*. Hier können *Tracks* miteinander verlinkt werden, wodurch man Interaktionen synchronisiert. Es werden zudem Höhe, Breite und Überschriften bestimmt und noch vieles mehr.

Wir erstellen nun einen simplen *Track* eines Scatter Plots (*mark_point( )*) von unseren Daten und erklären die Spalte *chromStart* als Input für die X-Achsenwerte. Damit der Track angezeigt wird, benutzen wir *view()*.

In [ ]:
track = gos.Track(data)
view = track.view()
print(view)

# Scatter plot mit "chromStart" als X-Achse
track.mark_point().encode(
    x=gos.X("chromStart:G"),
).view()

Der aufmerksame Tutorialnehmer hat womöglich Zweierlei bemerkt: \

Die **Chromosomen-Anzeige** oberhalb des Plots spiegelt die Chromosomen eines Menschen ab. In Gosling ist es möglich sechs Genome zu repräsentieren: \
*hg38, hg19, hg17, hg16, mm10, mm9*\
Die Anzeige kann auch durch *unknown* deaktiviert werden. Eingestellt wird die Anzeige mit der *assembly* Eigenschaft des *data_tracks* Instanz mithilfe der Funktion *gos.Assembly*.\
Leider ist unter den 6 möglichen Genomen nicht die Gerste zu finden...\

[Mehr Info](http://gosling-lang.org/docs/genome-builds/) \
[API](https://gosling-lang.github.io/gos/user_guide/generated/core/gosling.Assembly.html#gosling.Assembly)

Weiterhin wurde bei der Wertevergabe für die X-Achse *:G* zusätzlich deklariert. Dies ist die Kurzform der **Typ-Vergabe** für die Daten. Jeder Werte-Input muss mit einem *type* vergeben werden. Dies kann entweder durch das Anfügen von einem Doppelpunkt und der Kurzform nach dem Spaltennamen passieren oder durch ein weiteres Feld (Zu sehen im nächsten Codesegment).

| Datentyp | Kurzform | Eigenschaft |
| --------- | -------------- | ----------- | 
| quantitative | Q | Kontinuierliche realwertige Quantitäten | 
| nominal | N | Diskrete ungeordnete Kategorien |
| genomic | G | Genomische Basenposition |


In den nächsten Codesegmenten, werden die visuellen Unterschiede der Datentypen deutlich:

In [ ]:
# genomischer Datentyp für Y-Achse
track.mark_point().encode(
    x=gos.X("chromStart:G"), 
    y=gos.Y("chromStart:G", axis="left") # 'axis' gibt die Position der Achse im Plot an
).view()

In [ ]:
# quantitativer Datentyp für Y-Achse -> Wechsel der Diagonalen!
track.mark_point().encode(
    x=gos.X("chromStart:G"), 
    y=gos.Y("chromStart:Q", axis="left") # 'axis' gibt die Position der Achse im Plot an
).view()

In [ ]:
# nominaler Datentyp für Y-Achse -> Aufspaltung in die einzelnen Labels in 'Location'
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart:G"),
    y=gos.Y("Location:N") # Benutzung eines nominalen Feldes 'Location'
).view()

---

## Daten transformieren

Im nächsten Schritt möchten wir unsere Daten **transformieren**. Dies kann man mit mehreren Methoden:
- transform_filter()
- transform_filter_not()
- transform_exon_split()
- transform_coverage()
- transform_log()
- transform_str_concat()
- transform_str_replace() \

[Methoden](https://gosling-lang.github.io/gos/user_guide/generated/toplevel/gosling.Track.html#gosling.Track)\
[Wrapper](https://gosling-lang.github.io/gos/user_guide/API.html)\
[Liste an Filtern](http://gosling-lang.org/docs/data#data-transform)

Damit können wir zB nominale Werte aus Spalten wie *Chromosome* filtern, die Coverage über eine Spalte berechnen, usw. \

Wir filtern nun nach dem 1. Chromosom und möchten alle genomischen Varianten uns einzeigen lassen, außer die innerhalb von Codon-Regionen.

In [ ]:
# filter für chr1
transformed_track = track.transform_filter(field="Chromosome", oneOf=["chr1"])
# filtere codon Varianten raus
transformed_track = transformed_track.transform_filter_not(field="Location", oneOf=["codon"])

# zeige die gefilterten Daten
transformed_track.mark_point().encode(
    x=gos.X("chromStart", type="genomic"),
).view()

In [ ]:
# berechne coverage
coverage_data = track.transform_coverage(startField="chromStart", endField="chromEnd", newField="cov")

# zeige die coverage als line plot
coverage_data.mark_line().encode(
    y=gos.Y("cov:Q"),
    x=gos.X("chromStart:G")
).view()

---

## Erste Visualisierung (Hands On)

Du hast nun alle Infos, um folgende Aufgabe selbst zu visualisieren.

Benutze den Datensatz *data* und filtere ihn so, dass er nur noch Datenpunkte der Chromosomen 1, 3 und 5 besitzt. \
Dann filtere erneut nach "Splice" in der nominalen "Location"-Spalte unseres Datensatzes. \
Erstelle nun ein Scatter-Plot, welches die "Location" auf den genomischen Positionen darstellt. \
Vergesse nicht, dir den Track mit der *view( )* anzeigen zu lassen!

- benutze *data*
- filtere Chromosom 1,3,5
  - .transform_filter()
- filtere nach Spalte *Location* nach *splice* (Splice Sites)
  - .transform_filter()
- erstelle ein Scatter Plot
  - .mark_point()
- trage als X-Achse *chromStart* auf und als Y-Achse *Location* auf (achte auf die Types)
  - .encode()
  - gos.X()
  - gos.Y()
- lass dir deinen ersten Track darstellen
  - .view()

In [ ]:
your_track = ... # nehme den data Track oder erstelle dir einen neuen

your_tack = your_track.transform_filter( ... ) # filter nach Chr 1, 3, 5
your_tack = your_track.transform_filter( ... ) # filter nach splice Sites

your_track = your_track.mark_point().encode(
    x=gos.X( ... ), # setze X-Achse
    y=gos.Y( ... ), # setze Y-Achse
)

your_track.view() # visualisiere deinen track

---
---

Das Ende von Teiltutorial **01_Datenaufbereitung**. \
Hier findest du das nächste [Tutorial](https://colab.research.google.com/drive/1_-VCzxuDhxIDSPjx-S2mBZfkurWw7Dnr?usp=sharing).